# 02.00 Geospatial Analysis: Cleaning
## Issue 1279
---
[Neighborhood Council Shapefile Source](https://data.lacity.org/City-Infrastructure-Service-Requests/Neighborhood-Councils-Certified-/fu65-dz2f) | [LA Blocks Census Shapefile Source](https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/)

# Table of Contents
---
## [Package & Data Imports  ](#Package-&-Data-Imports)
## [Data Cleaning](#Cleaning)
## [Consistency](#Checking-Consistency-Between-gdf_nc-and-df)
## [Nulls and Duplicates](#Nulls-&-Duplicates)
## Spatial Joins
> ## [Spatial Join - Neighborhood Council and Blocks Geodata  ](#Spatial-Join---Neighborhood-Council-and-Blocks-Geodata)
> ## [Spatial Join - 311 Request Data (Address Points) with Block-Neighborhood Council Geodata  ](#Spatial-Join---311-Request-Data-(Address-Points)-with-Block-Neighborhood-Council-Geodata)

# Package & Data Imports
---

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# https://stackoverflow.com/questions/13440102/getting-bounding-box-of-city
import geopandas as gpd

# https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972
from shapely.geometry import Point, Polygon

from titlecase import titlecase

%matplotlib inline
import warnings
warnings.simplefilter("ignore")
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=DeprecationWarning)

In [2]:
# 311 Data 10/01/2021-10/01/2022 from API see 01 notebook
raw_df = pd.read_csv('./data/clean_01Oct21_01Oct22_api.csv')
df = raw_df.copy()

# https://geopandas.org/en/stable/getting_started/introduction.html
# NC boundaries: https://data.lacity.org/City-Infrastructure-Service-Requests/Neighborhood-Councils-Certified-/fu65-dz2f
raw_gdf_nc = gpd.read_file('./data/Neighborhood Councils (Certified)/geo_export_88bb18d9-f96c-4351-8be9-594f258ed0d3.shp')
gdf_nc = raw_gdf_nc.copy()

# Block boundaries: https://www2.census.gov/geo/tiger/TIGER2020PL/STATE/06_CALIFORNIA/06037/
# tl_2020_06037_tabblock20.zip
raw_gdf_blk = gpd.read_file("./data/tl_2020_06037_tabblock20/tl_2020_06037_tabblock20.shp")
gdf_blk = raw_gdf_blk.copy()

In [3]:
print(df.shape)
df.head(1)

(1042202, 16)


,requestId,createdDate,closedDate,typeId,typeName,address,latitude,longitude,agencyId,agencyName,sourceId,srnumber,sourceName,councilId,councilName,councilID
0,8610592,2021-10-01 00:01:14,2021-10-08 09:24:53,4,Bulky Items,"11614 N HERRICK AVE, 91340",34.281312,-118.425453,3,Sanitation Bureau,8,1-2079512481,Phone Call,64,Pacoima,64


In [4]:
print(gdf_nc.shape)
gdf_nc.head(1)

(99, 9)


,date_certi,time_certi,dwebsite,name,nc_id,objectid,service_re,waddress,geometry
0,2002-10-22,00:00:00.000,http://empowerla.org/ANC,ARLETA NC,6.0,1.0,REGION 1 - NORTH EAST VALLEY,http://www.arletanc.org/,"POLYGON ((-118.45005 34.24992, -118.45055 34.2..."


In [5]:
print(gdf_blk.shape)
gdf_blk.head(1)

(91626, 16)


,STATEFP20,COUNTYFP20,TRACTCE20,BLOCKCE20,GEOID20,NAME20,MTFCC20,UR20,UACE20,UATYPE20,FUNCSTAT20,ALAND20,AWATER20,INTPTLAT20,INTPTLON20,geometry
0,06,037,109300,2015,060371093002015,Block 2015,G5040,None,None,None,S,17913,0,+34.2673969,-118.4797070,"POLYGON ((-118.48022 34.26741, -118.48021 34.2..."


# Checking Consistency Between gdf_nc and df
#### To Note:
- df['councilID'] == gdf['objectiid']
- df contains a 0 ID for No Council

In [6]:
# gdf_nc

In [7]:
# gdf_nc.sort_values(by = 'objectid')

In [8]:
# df[df['councilId'] == 27]

In [9]:
# gdf_nc[gdf_nc['objectid'] == 27]

# Nulls & Duplicates

In [10]:
df.isnull().sum()

requestId      0
createdDate    0
closedDate     0
typeId         0
typeName       0
address        0
latitude       0
longitude      0
agencyId       0
agencyName     0
sourceId       0
srnumber       0
sourceName     0
councilId      0
councilName    0
councilID      0
dtype: int64

In [11]:
df[df['requestId'].duplicated() == True]

,requestId,createdDate,closedDate,typeId,typeName,address,latitude,longitude,agencyId,agencyName,sourceId,srnumber,sourceName,councilId,councilName,councilID


# Spatial Join - Neighborhood Council and Blocks Geodata

In [12]:
# Neighborhood Council
# get necessary columns; objectid = councilId
gdf_nc = gdf_nc[['name', 'objectid', 'geometry']].sort_values(by = 'objectid').reset_index(drop = True)

# format data
gdf_nc['name'] = gdf_nc['name'].apply(lambda x: titlecase(x.strip(' NC')))
gdf_nc['objectid'] = gdf_nc['objectid'].apply(lambda x: int(x))

print(gdf_nc.shape)
gdf_nc.head()

(99, 3)


,name,objectid,geometry
0,Arleta,1,"POLYGON ((-118.45005 34.24992, -118.45055 34.2..."
1,Arroyo Seco,2,"POLYGON ((-118.22325 34.10393, -118.22367 34.1..."
2,Atwater Village,3,"POLYGON ((-118.27576 34.15376, -118.26184 34.1..."
3,Bel Air-Beverly Crest,4,"POLYGON ((-118.47485 34.12634, -118.47411 34.1..."
4,Boyle Heights,5,"POLYGON ((-118.21439 34.06063, -118.21303 34.0..."


In [13]:
# Blocks
gdf_blk = gdf_blk[['GEOID20', 'geometry', 'NAME20', 'TRACTCE20', 'COUNTYFP20']]
print(gdf_blk.shape)
gdf_blk.head(1)

(91626, 5)


,GEOID20,geometry,NAME20,TRACTCE20,COUNTYFP20
0,060371093002015,"POLYGON ((-118.48022 34.26741, -118.48021 34.2...",Block 2015,109300,037


In [14]:
# confirming unique ID is 'GEOID20'
gdf_blk['GEOID20'].value_counts(ascending = False).head()

060371093002015    1
060379203421001    1
060379203261025    1
060379203031015    1
060379304002025    1
Name: GEOID20, dtype: int64

In [15]:
# spatial join of blocks and neighborhood councils
gdf_blk_nc = gpd.sjoin(gdf_blk, gdf_nc, how = 'inner', op = 'within')
gdf_blk_nc = gdf_blk_nc[['objectid', 'name', 'GEOID20', 'geometry']]
gdf_blk_nc['objectid'] = gdf_blk_nc['objectid'].apply(lambda x: int(x))
gdf_blk_nc.rename(columns = {'objectid': 'councilId'}, inplace = True)

In [16]:
print(gdf_blk_nc.shape)
gdf_blk_nc.head(1)

(23836, 4)


,councilId,name,GEOID20,geometry
0,27,Granada Hills North,060371093002015,"POLYGON ((-118.48022 34.26741, -118.48021 34.2..."


In [17]:
# check for duplicates
gdf_blk_nc[gdf_blk_nc['GEOID20'].duplicated() == True]

,councilId,name,GEOID20,geometry
52658,25,Foothill Trails District,060379800211009,"POLYGON ((-118.38622 34.26155, -118.38611 34.2..."
13230,56,Oho West,060371233041007,"POLYGON ((-118.40375 34.20120, -118.40350 34.2..."
53793,56,Oho West,060371233012003,"POLYGON ((-118.40255 34.19736, -118.40245 34.1..."
59584,56,Oho West,060371233043000,"POLYGON ((-118.40256 34.19394, -118.40233 34.1..."
78414,56,Oho West,060371233041008,"POLYGON ((-118.40375 34.20120, -118.40365 34.2..."
...,...,...,...,...
4886,43,La-32,060371993001012,"POLYGON ((-118.19193 34.08761, -118.19119 34.0..."
39649,43,La-32,060371993001007,"POLYGON ((-118.19289 34.08842, -118.19227 34.0..."
6051,16,East Hollywood,060371912011002,"POLYGON ((-118.29479 34.10012, -118.29471 34.1..."
82306,95,Wilmingto,060379800151002,"POLYGON ((-118.29448 33.78556, -118.29436 33.7..."


In [18]:
# deeper dive into duplicates - blocks are on borders of Neighborhood Councils
gdf_blk_nc[gdf_blk_nc['GEOID20'] == '060379800211009']

,councilId,name,GEOID20,geometry
52658,63,Pacoima,060379800211009,"POLYGON ((-118.38622 34.26155, -118.38611 34.2..."
52658,25,Foothill Trails District,060379800211009,"POLYGON ((-118.38622 34.26155, -118.38611 34.2..."


In [19]:
# Save request data with block IDs csv and shp
gdf_blk_nc.to_csv('data/clean_01Oct21_01Oct22_nc_blk.csv', index = False)
gdf_blk_nc.to_file('data/clean_01Oct21_01Oct22_nc_blk.geojson', driver='GeoJSON')
gdf_blk_nc.to_file('data/SHAPE-clean_01Oct21_01Oct22_nc_blk/clean_01Oct21_01Oct22_nc_blk.shp')

# Spatial Join - 311 Request Data (Address Points) with Block-Neighborhood Council Geodata

In [20]:
# https://towardsdatascience.com/geopandas-101-plot-any-data-with-a-latitude-and-longitude-on-a-map-98e01944b972

In [21]:
print(df.shape)
df.head(1)

(1042202, 16)


,requestId,createdDate,closedDate,typeId,typeName,address,latitude,longitude,agencyId,agencyName,sourceId,srnumber,sourceName,councilId,councilName,councilID
0,8610592,2021-10-01 00:01:14,2021-10-08 09:24:53,4,Bulky Items,"11614 N HERRICK AVE, 91340",34.281312,-118.425453,3,Sanitation Bureau,8,1-2079512481,Phone Call,64,Pacoima,64


In [22]:
# get points from longitude, latitude in df
geometry = [Point(xy) for xy in zip(df['longitude'], df['latitude'])]
geometry[:3]

In [23]:
# assign crs value and create GeoDataframe from 311 Requests df with Points geometry variable
crs = {'init': 'epsg:4326'}
df_geo = gpd.GeoDataFrame(df, crs = crs, geometry = geometry)

In [24]:
df_geo.shape

(1042202, 17)

In [25]:
# Check for duplicates (duplicates exist in original dataframe)
df_geo.drop_duplicates(subset = ['requestId']).shape

(1042202, 17)

In [26]:
# spatial join of addresses and blocks (and nc)
df_blk_nc = gpd.sjoin(df_geo, gdf_blk_nc, how="left", op='within')
df_blk_nc = df_blk_nc[['requestId', 'createdDate', 'closedDate', 'typeId', 'typeName',
       'address', 'councilId_left', 'councilName', 'geometry','GEOID20']]
df_blk_nc.rename(columns = {'councilId_left': 'councilId'}, inplace = True)
df_blk_nc.reset_index(drop = True, inplace = True)

In [27]:
print(f'df_blk_nc shape: {df_blk_nc.shape}')
print(f'df shape: {df.shape}')
df_blk_nc.head()

df_blk_nc shape: (1042400, 10)
df shape: (1042202, 17)


,requestId,createdDate,closedDate,typeId,typeName,address,councilId,councilName,geometry,GEOID20
0,8610592,2021-10-01 00:01:14,2021-10-08 09:24:53,4,Bulky Items,"11614 N HERRICK AVE, 91340",64,Pacoima,POINT (-118.42545 34.28131),060371043101005
1,8610569,2021-10-01 00:02:19,2021-10-08 13:36:33,5,Electronic Waste,"11614 N HERRICK AVE, 91340",64,Pacoima,POINT (-118.42545 34.28131),060371043101005
2,8610575,2021-10-01 00:03:11,2021-10-13 12:31:58,4,Bulky Items,"4539 N VISTA DEL MONTE AVE, 91403",72,Sherman Oaks,POINT (-118.45002 34.15416),060371413042005
3,8612855,2021-10-01 00:04:14,2021-10-05 14:25:02,6,Illegal Dumping,"918 W COLLEGE ST, 90012",38,Historic Cultural North,POINT (-118.24511 34.06647),060371977002002
4,8610587,2021-10-01 00:05:25,2021-10-08 14:16:32,7,Metal/Appliances,"11614 N HERRICK AVE, 91340",64,Pacoima,POINT (-118.42545 34.28131),060371043101005


## Duplicates

In [28]:
df_blk_nc[df_blk_nc['requestId'].duplicated() == True]

,requestId,createdDate,closedDate,typeId,typeName,address,councilId,councilName,geometry,GEOID20
30506,8677227,2021-10-08 16:42:13,2021-10-10 23:50:21,1,Graffiti,"5941 N WOODLEY AVE, 91436",24,Encino,POINT (-118.48160 34.17900),060379800241006
49023,8713144,2021-10-13 18:58:10,2021-10-13 19:00:29,1,Graffiti,"6300 N WOODLEY AVE, 91406",24,Encino,POINT (-118.48357 34.18496),060379800241007
54052,8726941,2021-10-15 11:06:11,2022-03-16 11:32:25,2,Homeless Encampment,"6300 N BALBOA BLVD, 91406",24,Encino,POINT (-118.50100 34.18439),060379800241003
56506,8733988,2021-10-16 14:33:08,2021-10-17 12:36:24,3,Animal Remains,"5436 W ZOO DR, 90027",3,Atwater Village,POINT (-118.30602 34.15556),060379800091014
56507,8733988,2021-10-16 14:33:08,2021-10-17 12:36:24,3,Animal Remains,"5436 W ZOO DR, 90027",3,Atwater Village,POINT (-118.30602 34.15556),060379800091014
...,...,...,...,...,...,...,...,...,...,...
1019715,10671139,2022-09-22 11:42:44,2022-09-29 23:23:30,2,Homeless Encampment,"6331 N WOODLEY AVE, 91406",24,Encino,POINT (-118.48409 34.18432),060379800241006
1020081,10672112,2022-09-22 13:07:56,2022-09-29 15:37:57,6,Illegal Dumping,"WOODLEY AVE AT BURBANK BLVD, 91436",24,Encino,POINT (-118.47560 34.17152),060379800241017
1021459,10675618,2022-09-22 18:10:32,2022-09-23 09:08:33,1,Graffiti,"6034 N BALBOA BLVD, 91406",24,Encino,POINT (-118.50100 34.18022),060379800241003
1023889,10684517,2022-09-23 15:47:06,2022-09-29 23:22:19,4,Bulky Items,"WOODLEY AVE AT BURBANK BLVD, 91436",24,Encino,POINT (-118.47560 34.17152),060379800241017


In [29]:
df_blk_nc[df_blk_nc['requestId']== 8677227]

,requestId,createdDate,closedDate,typeId,typeName,address,councilId,councilName,geometry,GEOID20
30505,8677227,2021-10-08 16:42:13,2021-10-10 23:50:21,1,Graffiti,"5941 N WOODLEY AVE, 91436",24,Encino,POINT (-118.48160 34.17900),060379800241006
30506,8677227,2021-10-08 16:42:13,2021-10-10 23:50:21,1,Graffiti,"5941 N WOODLEY AVE, 91436",24,Encino,POINT (-118.48160 34.17900),060379800241006


In [30]:
df_blk_nc.drop_duplicates(subset = ['requestId'], inplace = True)
print(df_blk_nc.shape)

(1042202, 10)


## Nulls

In [31]:
df_blk_nc.isnull().sum()

requestId           0
createdDate         0
closedDate          0
typeId              0
typeName            0
address             0
councilId           0
councilName         0
geometry            0
GEOID20        160710
dtype: int64

In [32]:
# Check for null less than 30%
df_blk_nc.isnull().sum().sum() / len(df_blk_nc)

0.15420235232709206

In [33]:
# drop nulls if less than 30%
df_blk_nc.dropna(inplace = True)

In [37]:
df_blk_nc.isnull().sum()

requestId      0
createdDate    0
closedDate     0
typeId         0
typeName       0
address        0
councilId      0
councilName    0
geometry       0
GEOID20        0
dtype: int64

## Save cleaned request set with block geodata

In [34]:
df_blk_nc['requestId'].value_counts(ascending = False)

8610592     1
10017300    1
10017304    1
10017307    1
10017329    1
           ..
9312932     1
9312913     1
9313502     1
9312987     1
10747580    1
Name: requestId, Length: 881492, dtype: int64

In [36]:
# Save request data with block IDs csv and shp
df_blk_nc.to_csv('data/clean_01Oct21_01Oct22_nc_blk_req.csv', index = False)
gdf_blk_nc.to_file('data/clean_01Oct21_01Oct22_nc_blk_req.geojson', driver='GeoJSON')
df_blk_nc.to_file('data/SHAPE-clean_01Oct21_01Oct22_nc_blk_req/clean_01Oct21_01Oct22_nc_blk_req.shp') 